In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pefile
import hashlib
import capstone
import re
import PyPDF2
import mimetypes
from tkinter.scrolledtext import ScrolledText
import threading
import os
from datetime import datetime


# Functie pentru calcularea hash-ului MD5
def calculate_hash(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

# Functie pentru extragerea string-urilor dintr-un fisier
def extract_strings(file_path, min_length=4):
    with open(file_path, "rb") as f:
        content = f.read()
    return re.findall(rb"[ -~]{%d,}" % min_length, content)

# Functie pentru extragerea informatiilor din fisiere PE (exe/dll)
def extract_pe_info(file_path):
    try:
        pe = pefile.PE(file_path)
        imports = []
        exports = []
        packer_info = ["Packer Info Not Available"]

        if hasattr(pe, "DIRECTORY_ENTRY_IMPORT"):
            for entry in pe.DIRECTORY_ENTRY_IMPORT:
                imports.append(entry.dll.decode() if entry.dll else "Unknown DLL")
                for imp in entry.imports:
                    imports.append(f"{entry.dll.decode()} -> {imp.name.decode() if imp.name else 'Unknown'}")

        if hasattr(pe, "DIRECTORY_ENTRY_EXPORT"):
            exports = [exp.name.decode() for exp in pe.DIRECTORY_ENTRY_EXPORT.symbols if exp.name]

        return imports, exports, packer_info
    except Exception as e:
        return ["Error reading PE imports"], ["Error reading PE exports"], [str(e)]

# Functie pentru extragerea codului de asamblare
def extract_assembly_code(file_path):
    try:
        pe = pefile.PE(file_path)
        if not hasattr(pe, "OPTIONAL_HEADER") or not hasattr(pe.OPTIONAL_HEADER, "AddressOfEntryPoint"):
            return ["Invalid PE file"]

        entry_point = pe.OPTIONAL_HEADER.AddressOfEntryPoint
        section = next((s for s in pe.sections if s.contains_rva(entry_point)), None)

        if not section:
            return ["No valid code section found"]

        code = section.get_data()
        md = capstone.Cs(capstone.CS_ARCH_X86, capstone.CS_MODE_32)
        disassembled = [f"{i.address:x}:\t{i.mnemonic}\t{i.op_str}" for i in md.disasm(code, entry_point)]

        return disassembled if disassembled else ["No assembly instructions found"]
    except Exception as e:
        return [f"Error: {e}"]

# Functie pentru extragerea textului din PDF
def extract_pdf_text(file_path):
    try:
        with open(file_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            text = "\n".join([page.extract_text() or "" for page in reader.pages])
        return text
    except Exception as e:
        return f"Error extracting PDF text: {e}"

# Functie pentru extragerea hex dump
def extract_hex_dump(file_path, size=256):
    with open(file_path, "rb") as f:
        content = f.read(size)
    return " ".join(f"{byte:02x}" for byte in content)

# Functie pentru analizarea fisierului
def analyze_file(file_path):
    threading.Thread(target=display_file_info, args=(file_path,), daemon=True).start()

# Functie pentru selectarea unui fisier
def browse_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        analyze_file(file_path)

# Functie pentru salvarea analizei in fisier text
def save_analysis(file_path, analysis_content):
    folder_name = "Analize_Fisiere"
    os.makedirs(folder_name, exist_ok=True)

    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    file_name = f"{folder_name}/Analiza_{timestamp}.txt"
    
    with open(file_name, "w", encoding="utf-8") as file:
        file.write(analysis_content)
    
    messagebox.showinfo("Success", f"Analiza a fost salvată în fișierul: {file_name}")

# Functie pentru afisarea informatiilor despre fisier
def display_file_info(file_path):
    try:
        file_extension = file_path.split('.')[-1].lower()
        file_type, _ = mimetypes.guess_type(file_path)
        file_hash = calculate_hash(file_path)
        hex_dump = extract_hex_dump(file_path)
        strings_extracted = extract_strings(file_path)

        result_text.delete(1.0, tk.END)
        result_text.tag_configure("title", foreground="blue", font=("Helvetica", 12, "bold"))
        result_text.tag_configure("section", foreground="green", font=("Helvetica", 10, "bold"))

        # Pregatire analiza completa
        analysis_content = f"File Analysis\nFile Path: {file_path}\nFile Type: {file_type or 'Unknown'}\n"
        analysis_content += f"File Hash (MD5): {file_hash}\n\nHex Dump (first 256 bytes):\n{hex_dump}\n\n"

        result_text.insert(tk.END, analysis_content, "title")

        # Extragerea string-urilor
        strings_display = "Extracted Strings (min 4 characters):\n" + "\n".join([s.decode(errors='ignore') for s in strings_extracted[:100]]) + "...\n\n"
        result_text.insert(tk.END, strings_display, "section")
        analysis_content += strings_display

        if file_extension == "txt":
            with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                text_content = f.read()
            text_display = f"Text File Content:\n{text_content[:500]}...\n\n"
            result_text.insert(tk.END, text_display, "section")
            analysis_content += text_display

        elif file_extension == "pdf":
            pdf_text = extract_pdf_text(file_path)
            pdf_display = f"PDF Content:\n{pdf_text[:500]}...\n\n"
            result_text.insert(tk.END, pdf_display, "section")
            analysis_content += pdf_display

        elif file_extension in ["exe", "dll"]:
            imports, exports, packer_info = extract_pe_info(file_path)
            assembly_code = extract_assembly_code(file_path)
            
            imports_display = "Imports:\n" + "\n".join(imports) + "\n\n"
            exports_display = "Exports:\n" + "\n".join(exports) + "\n\n"
            assembly_display = "Assembly Code:\n" + "\n".join(assembly_code[:100]) + "...\n\n"
            
            result_text.insert(tk.END, imports_display, "section")
            result_text.insert(tk.END, exports_display, "section")
            result_text.insert(tk.END, assembly_display, "section")
            
            analysis_content += imports_display + exports_display + assembly_display

        # Salvare analiza in fisier text
        save_analysis(file_path, analysis_content)

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

# Crearea interfetei grafice
root = tk.Tk()
root.title("File Analyzer")

browse_button = tk.Button(root, text="Browse File", command=browse_file)
browse_button.pack(pady=10)

result_text = ScrolledText(root, width=100, height=30, wrap=tk.WORD)
result_text.pack(padx=10, pady=10)

root.mainloop()
